In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Select device: prefer MPS on macOS, then CUDA, then CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available(): 
    device = torch.device("cuda")               
else:
    device = torch.device("cpu")

print("Using:", device)




Using: mps


In [4]:
# Use a German BERT model from Hugging Face
model_name = "dbmdz/bert-base-german-cased"

# Load tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4,  # adjust if your dataset has a different number of labels
    ignore_mismatched_sizes=True,
)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
import pandas as pd

df = pd.read_csv('../data/data_processed/all_data.csv')
df.head(100)

,filename,text,label
0,reminder_172.txt,letzte mahnung vor aussenstellung rechnung re...,reminder
1,reminder_166.txt,erste mahnung mahnung zur rechnung nr. re-8676...,reminder
2,reminder_199.txt,erste mahnung mahnung zur rechnung nr. re-7506...,reminder
3,order_138.txt,bestellung bestellnummer po-2024-3137 bestell...,order
4,order_110.txt,dringende bestellung - eilt bestellnummer ei...,order
...,...,...,...
95,contract_132.txt,software-lizenzvertrag zwischen bauer manufact...,contract
96,complaint_72.txt,qualitätsreklamation lieferung vom 11.11.2025...,complaint
97,contract_5.txt,liefervertrag zwischen koch engineering käuf...,contract
98,invoice_58.txt,koch engineering gartenstraße 12 frankfurt am ...,invoice


In [6]:
df = df[['text','label']]
df.head()

,text,label
0,letzte mahnung vor aussenstellung rechnung re...,reminder
1,erste mahnung mahnung zur rechnung nr. re-8676...,reminder
2,erste mahnung mahnung zur rechnung nr. re-7506...,reminder
3,bestellung bestellnummer po-2024-3137 bestell...,order
4,dringende bestellung - eilt bestellnummer ei...,order


In [7]:
df=df.dropna()

In [8]:
from sklearn.preprocessing import LabelEncoder
# Initialize the label encoder
label_encoder = LabelEncoder()
# Encode the labels
df['label'] = label_encoder.fit_transform(df['label'])
df.head()

,text,label
0,letzte mahnung vor aussenstellung rechnung re...,4
1,erste mahnung mahnung zur rechnung nr. re-8676...,4
2,erste mahnung mahnung zur rechnung nr. re-7506...,4
3,bestellung bestellnummer po-2024-3137 bestell...,3
4,dringende bestellung - eilt bestellnummer ei...,3


In [9]:
from sklearn.model_selection import train_test_split
# Step 4: Split into train, validation, test sets (80 / 10 / 10)
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42)
# No further splitting here — we already have pre-split train/test CSVs
print(f'Training set size: {len(train_df)}')
print(f'Testing set size: {len(test_df)}')
print(f'Validating set size: {len(val_df)}')

Training set size: 800
Testing set size: 100
Validating set size: 100


In [10]:
from datasets import DatasetDict, Dataset
# convert the merged dataframe to a dictionary
data_train_dict = train_df.to_dict('list')
dataset_train = Dataset.from_dict(data_train_dict)
data_train_dict = test_df.to_dict('list')
dataset_test = Dataset.from_dict(data_train_dict)
data_train_dict = val_df.to_dict('list')
dataset_validation = Dataset.from_dict(data_train_dict)
# create a dataset dictionary
dataset = DatasetDict({'train': dataset_train,'test':dataset_test,'validation':dataset_validation})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
})

In [11]:
text_p = []
max_length=512
def tokenize_data(example):
    try:
        return tokenizer(str(example['text']),
                         padding='max_length',
                         truncation=True,
                         max_length=max_length,
                      )
    #truncation=True, padding=True ,max_length=128, return_overflowing_tokens=True,
    except:
        print("Error")

In [12]:
dataset = dataset.map(tokenize_data ,remove_columns=["text"])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
from transformers import TrainingArguments
model_path = "./../models/bert_finetuned_faker_dataset"
training_args = TrainingArguments(
    model_path,
    num_train_epochs=3,                    # Better results
    save_total_limit=1,
    learning_rate=3e-5,
    save_strategy="epoch",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,         # Eval can handle more
    gradient_accumulation_steps=2,         # Effective batch_size=16
    dataloader_num_workers=0,              # Simpler, fewer bugs
    fp16=False,
    logging_steps=50,
    report_to="none"
)

In [14]:
#number 32218
train_dataset = dataset['train']
eval_dataset = dataset['test']


In [15]:
import numpy as np
from evaluate import load
from sklearn.metrics import precision_recall_fscore_support

metric = load("accuracy")

preds = []

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    preds.append(eval_pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true=labels, y_pred=predictions, average='weighted'
    )

    accuracy = metric.compute(predictions=predictions, references=labels)

    return {
        'accuracy': accuracy['accuracy'],  # <- note: dict from `evaluate`
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

Using the latest cached version of the module from /Users/harsh/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Tue Nov  4 03:33:53 2025) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
    
)


In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.233400,0.005433,0.800000,0.666963,0.800000,0.720128
2,0.004400,0.002326,0.800000,0.666963,0.800000,0.720128
3,0.002800,0.001974,0.800000,0.666963,0.800000,0.720128


/Users/harsh/anaconda3/envs/doc-classifier-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/harsh/anaconda3/envs/doc-classifier-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/harsh/anaconda3/envs/doc-classifier-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

TrainOutput(global_step=150, training_loss=0.08020662198464076, metrics={'train_runtime': 361.2726, 'train_samples_per_second': 6.643, 'train_steps_per_second': 0.415, 'total_flos': 631477872230400.0, 'train_loss': 0.08020662198464076, 'epoch': 3.0})